In [14]:
# install PyGmsh
import sys
!{sys.executable} -m pip install -q -U python3-gmsh
import gmsh

ERROR: Could not find a version that satisfies the requirement python3-gmsh (from versions: none)
ERROR: No matching distribution found for python3-gmsh


ModuleNotFoundError: No module named 'gmsh'

In [12]:
!{sys.executable} -m pip install -q -U pygmsh
import pygmsh

ModuleNotFoundError: No module named 'gmsh'